In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, svm, tree
from sklearn.ensemble import RandomForestRegressor
from  sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [2]:
data = pd.read_csv("communities.csv")

In [3]:
def prepare_data(exclude_columns):
    data = pd.read_csv("communities.csv")
    data = data.drop(exclude_columns, axis=1)

    # One hot encoding for state column.
    data = pd.get_dummies(data, columns=["state"])

    x = data.drop('ViolentCrimesPerPop', axis=1)
    
    y = data['ViolentCrimesPerPop']

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.25, random_state=42)
    return x_train, x_test, y_train, y_test

In [4]:
def scale_data(x_train, x_test, scaler=StandardScaler):

    scaler = scaler()
    scaler.fit(x_train)

    x_train_scaled = scaler.transform(x_train)
    x_train_scaled = pd.DataFrame(x_train_scaled, index=x_train.index, columns=x_train.columns)

    x_test_scaled = scaler.transform(x_test)
    x_test_scaled = pd.DataFrame(x_test_scaled, index=x_test.index, columns=x_test.columns)

    x_train_scaled.fillna(x_train_scaled.mean(), inplace=True)
    x_test_scaled.fillna(x_test_scaled.mean(), inplace=True)

    return x_train_scaled, x_test_scaled



In [5]:
def evaluate_regression(y_true_train, y_pred_train, y_true_test, y_pred_test, model_name=""):   
    return pd.DataFrame.from_records([[metrics.mean_squared_error(y_true_train, y_pred_train, squared=False),
                                       metrics.mean_squared_error(y_true_train, y_pred_train),
                                       metrics.mean_absolute_error(y_true_train, y_pred_train),
                                       metrics.max_error(y_true_train, y_pred_train),
                                       metrics.r2_score(y_true_train, y_pred_train), 
                                       metrics.mean_squared_error(y_true_test, y_pred_test,  squared=False),
                                      metrics.mean_squared_error(y_true_test, y_pred_test),
                                       metrics.mean_absolute_error(y_true_test, y_pred_test),
                                       metrics.max_error(y_true_test, y_pred_test),
                                       metrics.r2_score(y_true_test, y_pred_test)]], 
                                     
                                     index=[model_name], 
                                     columns=['RMSE_train','mean_squared_error_train', 'mean_absolute_error_train', 'max_error_train', "r2_score_train",
                                         'RMSE_test', 'mean_squared_error_test', 'mean_absolute_error_test', 'max_error_test', "r2_score_test"])


In [6]:
x_train, x_test, y_train, y_test = prepare_data(exclude_columns=["county",
                                                                "community",
                                                                "communityname",
                                                                ])
x_train, x_test = scale_data(x_train, x_test, scaler=StandardScaler)

In [7]:
results = pd.DataFrame()
for model in [linear_model.LinearRegression(), 
            linear_model.RidgeCV(),
            linear_model.LassoCV(),
            svm.SVR(kernel="linear"),
            svm.SVR(kernel="rbf"),
            svm.SVR(kernel="poly"),
            tree.DecisionTreeRegressor()]:
    model.fit(x_train, y_train)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    results = results.append(evaluate_regression(y_train, y_train_pred, y_test, y_test_pred,
                        model_name=model))


In [8]:
results.sort_values("RMSE_test")

,RMSE_train,mean_squared_error_train,mean_absolute_error_train,max_error_train,r2_score_train,RMSE_test,mean_squared_error_test,mean_absolute_error_test,max_error_test,r2_score_test
LassoCV(),1.279265e-01,1.636518e-02,8.905247e-02,7.660781e-01,0.707599,1.238333e-01,1.533470e-02,8.674278e-02,6.400473e-01,6.846289e-01
"RidgeCV(alphas=array([ 0.1, 1. , 10. ]))",1.199164e-01,1.437995e-02,8.486875e-02,7.277727e-01,0.743069,1.269664e-01,1.612047e-02,9.103543e-02,6.857531e-01,6.684688e-01
SVR(kernel='linear'),1.221549e-01,1.492181e-02,8.679873e-02,7.962276e-01,0.733388,1.302102e-01,1.695471e-02,9.257628e-02,6.984992e-01,6.513121e-01
SVR(),8.488995e-02,7.206303e-03,7.032034e-02,5.556636e-01,0.871243,1.376818e-01,1.895628e-02,1.010344e-01,7.318555e-01,6.101480e-01
SVR(kernel='poly'),8.977506e-02,8.059561e-03,7.402678e-02,6.079991e-01,0.855998,1.480762e-01,2.192657e-02,1.066338e-01,8.748978e-01,5.490615e-01
DecisionTreeRegressor(),1.805790e-18,3.260876e-36,2.019009e-19,2.775558e-17,1.000000,1.943547e-01,3.777375e-02,1.307615e-01,8.500000e-01,2.231509e-01
LinearRegression(),1.190710e-01,1.417790e-02,8.474138e-02,7.100313e-01,0.746679,2.025307e+08,4.101870e+16,6.537607e+07,4.314820e+09,-8.435842e+17


In [9]:
print("Reference RMSE", results.sort_values("RMSE_test").iloc[0]["RMSE_test"])

Reference RMSE 0.12383334723075583


# Neural Network

In [10]:
from keras import models
from keras import layers
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense

In [11]:
x_train.shape

(1495, 168)

In [22]:
import plotly.graph_objects as go
def plot_hist(history):
    fig = go.Figure()
    fig.add_trace(go.Scattergl(y=history.history['loss'],
                        name='Train'))
    fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                        name='Valid'))
    fig.update_layout(height=500, width=700,
                    xaxis_title='Epoch',
                    yaxis_title='Loss')
    fig.show()

In [24]:
def baseline_model():
    model = Sequential()
    model.add(Dense(2, input_dim=168,  activation='relu'))
    model.add(Dense(3, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [38]:
def deep_model():
    model = Sequential()
    model.add(Dense(12, input_dim=168,  activation='sigmoid'))
    model.add(Dense(12, activation='sigmoid'))
    model.add(Dense(6, activation='sigmoid'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()

    return model

In [27]:
model = baseline_model()
history = model.fit(x_train, y_train, epochs=20, validation_split=0.2, verbose=0)
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

results = results.append(evaluate_regression(y_train, y_train_pred, y_test, y_test_pred,
                        model_name="Keras:baseline_model 2:3:1"))
plot_hist(history)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 2)                 338       
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 9         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 4         
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________


In [40]:
model = deep_model()
history = model.fit(x_train, y_train, epochs=20, validation_split=0.2, verbose=0)
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

results = results.append(evaluate_regression(y_train, y_train_pred, y_test, y_test_pred,
                        model_name="Keras:deep_model 12:12:6 all_sigmoid"))
plot_hist(history)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 12)                2028      
_________________________________________________________________
dense_36 (Dense)             (None, 12)                156       
_________________________________________________________________
dense_37 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 7         
Total params: 2,269
Trainable params: 2,269
Non-trainable params: 0
_________________________________________________________________


In [41]:
results.sort_values("RMSE_test")

,RMSE_train,mean_squared_error_train,mean_absolute_error_train,max_error_train,r2_score_train,RMSE_test,mean_squared_error_test,mean_absolute_error_test,max_error_test,r2_score_test
LassoCV(),1.279265e-01,1.636518e-02,8.905247e-02,7.660781e-01,0.707599,1.238333e-01,1.533470e-02,8.674278e-02,6.400473e-01,6.846289e-01
Keras:deep_model 12:12:6 all_sigmoid,1.188157e-01,1.411718e-02,7.939493e-02,7.596041e-01,0.747764,1.247719e-01,1.556802e-02,8.515799e-02,7.041526e-01,6.798304e-01
Keras:deep_model 6:12:6 sigmoid,1.192710e-01,1.422558e-02,8.332297e-02,6.741686e-01,0.745827,1.266157e-01,1.603153e-02,8.971219e-02,6.515012e-01,6.702980e-01
"RidgeCV(alphas=array([ 0.1, 1. , 10. ]))",1.199164e-01,1.437995e-02,8.486875e-02,7.277727e-01,0.743069,1.269664e-01,1.612047e-02,9.103543e-02,6.857531e-01,6.684688e-01
Keras:deep_model 12:12:6 sigmoid,1.205101e-01,1.452267e-02,8.130825e-02,7.107195e-01,0.740519,1.289130e-01,1.661855e-02,8.870923e-02,7.807262e-01,6.582254e-01
SVR(kernel='linear'),1.221549e-01,1.492181e-02,8.679873e-02,7.962276e-01,0.733388,1.302102e-01,1.695471e-02,9.257628e-02,6.984992e-01,6.513121e-01
Keras:baseline_model 2:3:1,1.199998e-01,1.439994e-02,8.043026e-02,9.231929e-01,0.742712,1.321630e-01,1.746706e-02,8.993189e-02,7.291537e-01,6.407750e-01
Keras:deep_model 12:12:6 sigmoid,1.090428e-01,1.189034e-02,7.212627e-02,7.301340e-01,0.787552,1.374419e-01,1.889028e-02,9.019274e-02,7.862707e-01,6.115053e-01
SVR(),8.488995e-02,7.206303e-03,7.032034e-02,5.556636e-01,0.871243,1.376818e-01,1.895628e-02,1.010344e-01,7.318555e-01,6.101480e-01
Keras:deep_model 12:12:6 sigmoid,1.447287e-01,2.094641e-02,9.886027e-02,7.336865e-01,0.625745,1.411657e-01,1.992776e-02,9.904748e-02,6.363380e-01,5.901688e-01


In [54]:
print("Best NN test RMSE", results.loc["Keras:deep_model 12:12:6 all_sigmoid", "RMSE_test"], " \nReference    RMSE 0.12383334723075583")


Best NN test RMSE 0.1247718767093575  
Reference    RMSE 0.12383334723075583
